# 컴피티션 개인전
- 17시30분 마감
- 평가지표: AUC
- 1~3등에게 스타벅스 쿠폰을 전달

In [1]:
%pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 3.9 MB/s eta 0:00:00


In [2]:
%pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.1 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import KFold, cross_val_score
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
# from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import optuna

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [4]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 데이터 경로와 시드값

In [5]:
DATA_PATH  = "/content/drive/MyDrive/the_datas/data/"
SEED = 42

# 데이터 불러오기

In [6]:
train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터
train_tr.shape ,test_tr.shape, train_target.shape, submit.shape

((523105, 7), (441196, 7), (14940, 2), (12225, 2))

In [7]:
train_ft = pd.read_csv(f"{DATA_PATH}train_ft_군집분석_34_김이지.csv") # 학습 데이터(피처)
test_ft = pd.read_csv(f"{DATA_PATH}test_ft_군집분석_34_김이지.csv") # 테스트 데이터(피처)

train_ft.shape , test_ft.shape

((14940, 488), (12225, 488))

In [8]:
# display(train.head())
# display(train.info())

- 공통 피처 파일 불러오기

In [9]:
# train_ft = pd.read_csv(f"{DATA_PATH}train_common_3.csv") # 학습 데이터(피처)
# test_ft = pd.read_csv(f"{DATA_PATH}test_common_3.csv") # 테스트 데이터(피처)

# train_ft.shape , test_ft.shape

# 결측치 처리

In [10]:
mask = train_ft.isnull().sum() > 0
train_ft.isnull().sum()[mask]

,0
3회이상count,4214
구매금액표준편차,388
구매금액왜도,840
구매금액첨도,1265


In [11]:
mask = test_ft.isnull().sum() > 0
test_ft.isnull().sum()[mask]

,0
3회이상count,3284
구매금액표준편차,242
구매금액왜도,599
구매금액첨도,954


In [12]:
train_ft["3회이상count"] = train_ft["3회이상count"].fillna(0)
train_ft["구매금액표준편차"] = train_ft["구매금액표준편차"].fillna(0)
train_ft["구매금액왜도"] = train_ft["구매금액왜도"].fillna(0)
train_ft["구매금액첨도"] = train_ft["구매금액첨도"].fillna(0)

test_ft["3회이상count"] = test_ft["3회이상count"].fillna(0)
test_ft["구매금액표준편차"] = test_ft["구매금액표준편차"].fillna(0)
test_ft["구매금액왜도"] = test_ft["구매금액왜도"].fillna(0)
test_ft["구매금액첨도"] = test_ft["구매금액첨도"].fillna(0)

In [13]:
train_ft.isnull().sum().sum(), test_ft.isnull().sum().sum()

(0, 0)

# 특성 공학(Feature Engineering)

- ID 변수 제외

In [14]:
train_ft = train_ft.iloc[:,1:]
test_ft = test_ft.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 487), (12225, 487))

- 추가 피처 만들어 보기

In [15]:
train_ft

,구매횟수,내점일수,구매주기,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,12시이전구매비율,12시이후_18시이전구매비율,...,브랜드_cluster_pivot_횟수비율24,브랜드_cluster_pivot_횟수비율25,브랜드_cluster_pivot_횟수비율26,브랜드_cluster_pivot_횟수비율27,브랜드_cluster_pivot_횟수비율28,브랜드_cluster_pivot_횟수비율29,브랜드_cluster_pivot_횟수비율30,브랜드_cluster_pivot_횟수비율31,브랜드_cluster_pivot_횟수비율32,브랜드_cluster_pivot_횟수비율33
0,16,11,23,0.050000,0.250000,0.400000,0.300000,3,0.300000,0.450000,...,0.000000,0.0,0.062500,0.0,0.0,0.000000,0.000000,0.0,0.125,0.000000
1,41,21,16,0.357143,0.166667,0.357143,0.119048,3,0.214286,0.476190,...,0.121951,0.0,0.024390,0.0,0.0,0.000000,0.000000,0.0,0.000,0.000000
2,102,56,6,0.464912,0.140351,0.175439,0.219298,0,0.122807,0.728070,...,0.157895,0.0,0.052632,0.0,0.0,0.052632,0.000000,0.0,0.000,0.000000
3,191,92,3,0.379147,0.180095,0.236967,0.203791,3,0.161137,0.616114,...,0.750000,0.1,0.600000,0.0,0.0,0.000000,0.000000,0.0,0.000,0.000000
4,55,27,11,0.112903,0.612903,0.209677,0.064516,4,0.096774,0.741935,...,0.047619,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14935,16,10,24,0.000000,0.578947,0.052632,0.368421,2,0.263158,0.578947,...,0.000000,0.0,0.100000,0.1,0.0,0.000000,0.000000,0.2,0.000,0.000000
14936,8,8,29,0.125000,0.625000,0.250000,0.000000,0,0.000000,0.500000,...,0.000000,0.0,0.019231,0.0,0.0,0.000000,0.000000,0.0,0.000,0.019231
14937,22,12,26,0.208333,0.458333,0.208333,0.125000,1,0.375000,0.541667,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.022222,0.0,0.000,0.000000
14938,31,12,28,0.393939,0.393939,0.212121,0.000000,3,0.151515,0.484848,...,0.050000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000,0.000000


In [16]:
cols = [ col for col in train_ft.columns if col.startswith("중_pivot_횟수비율_") ]

In [17]:
# cols

In [18]:
# train_ft["중분류별_구매횟수_std"] = train_ft[cols].std(axis=1)
# train_ft["중분류별_구매횟수_skew"] = train_ft[cols].skew(axis=1)
# train_ft["중분류별_구매횟수_kurt"] = train_ft[cols].kurt(axis=1)

# test_ft["중분류별_구매횟수_std"] = test_ft[cols].std(axis=1)
# test_ft["중분류별_구매횟수_skew"] = test_ft[cols].skew(axis=1)
# test_ft["중분류별_구매횟수_kurt"] = test_ft[cols].kurt(axis=1)

# train_ft.shape, test_ft.shape

In [19]:
# 최소 구매금액범주화
# 중분류 개수

## Feature Encoding

In [20]:
cols = train_ft.select_dtypes("object").columns.tolist()
train_ft[cols].nunique()

,0
주구매지점,4
최대구매액_대분류,28
최소구매액_대분류,28


In [21]:
%pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 2.0 MB/s eta 0:00:00


In [22]:
import category_encoders as ce

In [23]:
enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft[["주구매지점"]])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft[["주구매지점"]])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 491), (12225, 491))

In [24]:
enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft[["최소구매액_대분류"]])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft[["최소구매액_대분류"]])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 519), (12225, 519))

In [25]:
enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft[["최대구매액_대분류"]])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft[["최대구매액_대분류"]])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 547), (12225, 547))

In [26]:
# enc = ce.count.CountEncoder()
# train_ft["최소구매액_대분류_cnt"] = enc.fit_transform(train_ft[["최소구매액_대분류"]])
# test_ft["최소구매액_대분류_cnt"] = enc.transform(test_ft[["최소구매액_대분류"]])

# train_ft.shape, test_ft.shape

In [27]:
# enc = ce.count.CountEncoder()
# train_ft["최대구매액_대분류_cnt"] = enc.fit_transform(train_ft[["최대구매액_대분류"]])
# test_ft["최대구매액_대분류_cnt"] = enc.transform(test_ft[["최대구매액_대분류"]])

# train_ft.shape, test_ft.shape

- 문자열 피처 삭제

In [28]:
cols

['주구매지점', '최대구매액_대분류', '최소구매액_대분류']

In [29]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)
train_ft.shape, test_ft.shape

((14940, 544), (12225, 544))

In [30]:
train_ft.select_dtypes("object").columns , test_ft.select_dtypes("object").columns

(Index([], dtype='object'), Index([], dtype='object'))

## Feature Scaling

In [31]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [32]:
train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)
train_ft.head()

,구매횟수,내점일수,구매주기,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,12시이전구매비율,12시이후_18시이전구매비율,...,최대구매액_대분류_19,최대구매액_대분류_20,최대구매액_대분류_21,최대구매액_대분류_22,최대구매액_대분류_23,최대구매액_대분류_24,최대구매액_대분류_25,최대구매액_대분류_26,최대구매액_대분류_27,최대구매액_대분류_28
0,-0.437126,-0.369867,0.002987,-1.029777,0.001191,0.838272,0.338186,0.109631,0.718557,-0.654150,...,-0.193649,-0.131248,-0.07785,-0.1402,-0.200973,-0.090737,-0.071977,-0.146044,-0.067118,-0.050497
1,0.239394,0.144110,-0.356452,0.323951,-0.390607,0.620171,-0.552996,0.109631,0.269465,-0.542415,...,-0.193649,-0.131248,-0.07785,-0.1402,-0.200973,-0.090737,-0.071977,-0.146044,-0.067118,-0.050497
2,1.890101,1.943028,-0.869935,0.798943,-0.514333,-0.304527,-0.059266,-1.643370,-0.209830,0.532172,...,-0.193649,-0.131248,-0.07785,-0.1402,-0.200973,-0.090737,-0.071977,-0.146044,-0.067118,-0.050497
3,4.298510,3.793345,-1.023980,0.420933,-0.327474,0.008592,-0.135636,0.109631,-0.009001,0.054536,...,-0.193649,-0.131248,-0.07785,-0.1402,-0.200973,-0.090737,-0.071977,-0.146044,-0.067118,-0.050497
4,0.618244,0.452496,-0.613193,-0.752532,1.707410,-0.130285,-0.821561,0.693965,-0.346226,0.591326,...,-0.193649,-0.131248,-0.07785,-0.1402,-0.200973,-0.090737,-0.071977,-0.146044,-0.067118,-0.050497


# 정답 데이터

In [33]:
target = train_target["target"]
target

,target
0,1.0
1,1.0
2,0.0
3,0.0
4,0.0
...,...
14935,0.0
14936,0.0
14937,0.0
14938,1.0


# 모델 튜닝하기

In [34]:
class RF_Objective:
    def __init__(self, x, y, seed):
        self.x = x
        self.y = y
        self.seed = seed
        self.cv = KFold(n_splits=5, shuffle=True, random_state=self.seed)

    def __call__(self, trial):
        hp = {
              'n_estimators': trial.suggest_int('n_estimators', 100, 1001),
              'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
              'max_depth': trial.suggest_int('max_depth', 3, 41),
              'min_samples_split': trial.suggest_int('min_samples_split', 6, 30),
              'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None])
        }
        model = RandomForestClassifier(random_state=self.seed, **hp)
        scores = cross_val_score(model, self.x, self.y, cv=self.cv, scoring='f1_macro', n_jobs=-1)
        return scores.mean()

class XG_Objective:
    def __init__(self, x, y, seed):
        self.x = x
        self.y = y
        self.seed = seed
        self.cv = KFold(n_splits=5, shuffle=True, random_state=self.seed)

    def __call__(self, trial):
        hp = {
            'n_estimators': trial.suggest_int('n_estimators', 50,201),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.31, step=0.01),
            'max_depth': trial.suggest_int('max_depth', 3, 12),
            'gamma': trial.suggest_float('gamma', 0.1, 5.1, step=0.1),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0, step=0.1),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0, step=0.1)
            }
        model = XGBClassifier(random_state=self.seed, **hp)
        scores = cross_val_score(model, self.x, self.y, cv=self.cv, scoring='f1_macro', n_jobs=-1)
        return scores.mean()

class LR_Objective:
    def __init__(self, x, y, seed):
        self.x = x
        self.y = y
        self.seed = seed
        self.cv = KFold(n_splits=5, shuffle=True, random_state=self.seed)

    def __call__(self, trial):
        hp = {
            'C': trial.suggest_float('C', 0.001, 10, step=0.001),
            'penalty': trial.suggest_categorical('penalty', ['l2',  None]),
            }
        model = LogisticRegression(random_state=self.seed, **hp)
        scores = cross_val_score(model, self.x, self.y, cv=self.cv, scoring='f1_macro', n_jobs=-1)
        return scores.mean()

class LGB_Objective:
    def __init__(self, x, y, seed):
        self.x = x
        self.y = y
        self.seed = seed
        self.cv = KFold(n_splits=5, shuffle=True, random_state=self.seed)

    def __call__(self, trial):
        hp = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 501),
            'learning_rate': trial.suggest_float('learning_rate',0.01, 0.31, step=0.01),
            'max_depth': trial.suggest_int('max_depth', 3, 16),
            'num_leaves': trial.suggest_int('num_leaves', 20,101),
            'min_child_samples': trial.suggest_int('min_child_samples', 5,31),
            }
        model = LGBMClassifier(random_state=self.seed, **hp)
        scores = cross_val_score(model, self.x, self.y, cv=self.cv, scoring='f1_macro', n_jobs=-1)
        return scores.mean()

In [35]:
sampler = optuna.samplers.TPESampler(seed=SEED) # 대체 모델

In [36]:
rf_study = optuna.create_study(
    direction='maximize',
    sampler= sampler
)

[I 2024-11-14 10:47:09,883] A new study created in memory with name: no-name-0cc2d790-c39f-4e13-a829-c0a4fa2f08fd


In [37]:
rf_objective_func = RF_Objective(train_ft, target, SEED)
rf_study.optimize(rf_objective_func, n_trials=10)

[W 2024-11-14 10:52:37,974] Trial 0 failed with parameters: {'n_estimators': 437, 'criterion': 'gini', 'max_depth': 26, 'min_samples_split': 9, 'max_features': None} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-34-ad2f7da7b6a5>", line 17, in __call__
    scores = cross_val_score(model, self.x, self.y, cv=self.cv, scoring='f1_macro', n_jobs=-1)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 712, in cross_val_score
    cv_results = cross_validate(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/py

KeyboardInterrupt: 

In [ ]:
rf_study.best_value, rf_study.best_trial.params

In [ ]:
lr_study = optuna.create_study(
    direction='maximize',
    sampler= sampler
)

[I 2024-11-12 03:39:51,029] A new study created in memory with name: no-name-2a7d4960-6c12-4834-9527-504f0f1aa33c


In [ ]:
mask = train_ft.isna().sum() > 0
train_ft.isna().sum()[mask]

,0


In [ ]:
lr_objective_func = LR_Objective(train_ft, target, SEED)
lr_study.optimize(lr_objective_func, n_trials=200)

[I 2024-11-12 03:40:04,053] Trial 0 finished with value: 0.690972824354945 and parameters: {'C': 3.746, 'penalty': 'l2'}. Best is trial 0 with value: 0.690972824354945.
[I 2024-11-12 03:40:11,672] Trial 1 finished with value: 0.6911569197568891 and parameters: {'C': 5.987, 'penalty': 'l2'}. Best is trial 1 with value: 0.6911569197568891.
[I 2024-11-12 03:40:17,337] Trial 2 finished with value: 0.6914464002374353 and parameters: {'C': 0.581, 'penalty': 'l2'}. Best is trial 2 with value: 0.6914464002374353.
[I 2024-11-12 03:40:25,852] Trial 3 finished with value: 0.6914750490825103 and parameters: {'C': 7.081, 'penalty': None}. Best is trial 3 with value: 0.6914750490825103.
[I 2024-11-12 03:40:31,871] Trial 4 finished with value: 0.6913236145783025 and parameters: {'C': 8.325, 'penalty': 'l2'}. Best is trial 3 with value: 0.6914750490825103.
[I 2024-11-12 03:40:39,971] Trial 5 finished with value: 0.6914750490825103 and parameters: {'C': 1.835, 'penalty': None}. Best is trial 3 with val

In [ ]:
lr_study.best_value, lr_study.best_trial.params

(0.6915735318924592, {'C': 5.266, 'penalty': 'l2'})

In [38]:
xg_study = optuna.create_study(
    direction='maximize',
    sampler= sampler
)

[I 2024-11-14 10:52:45,412] A new study created in memory with name: no-name-61b56613-2c52-40c7-97a7-15b0c1ad6f9a


In [39]:

xg_objective_func = XG_Objective(train_ft, target, SEED)
xg_study.optimize(xg_objective_func, n_trials=150)

[I 2024-11-14 10:53:21,353] Trial 0 finished with value: 0.7157871888510878 and parameters: {'n_estimators': 141, 'learning_rate': 0.22, 'max_depth': 3, 'gamma': 5.0, 'subsample': 0.9, 'colsample_bytree': 0.6}. Best is trial 0 with value: 0.7157871888510878.
[I 2024-11-14 10:54:03,541] Trial 1 finished with value: 0.7085214098238998 and parameters: {'n_estimators': 77, 'learning_rate': 0.060000000000000005, 'max_depth': 6, 'gamma': 2.7, 'subsample': 0.7, 'colsample_bytree': 0.6}. Best is trial 0 with value: 0.7157871888510878.
[I 2024-11-14 10:54:56,859] Trial 2 finished with value: 0.7129203852859771 and parameters: {'n_estimators': 143, 'learning_rate': 0.05, 'max_depth': 5, 'gamma': 1.9000000000000001, 'subsample': 0.7, 'colsample_bytree': 0.9}. Best is trial 0 with value: 0.7157871888510878.
[I 2024-11-14 10:55:55,375] Trial 3 finished with value: 0.7054409850498704 and parameters: {'n_estimators': 80, 'learning_rate': 0.16, 'max_depth': 8, 'gamma': 0.30000000000000004, 'subsample'

In [40]:
xg_study.best_value, xg_study.best_trial.params

(0.7207173157280883,
 {'n_estimators': 180,
  'learning_rate': 0.06999999999999999,
  'max_depth': 4,
  'gamma': 1.8000000000000003,
  'subsample': 0.6,
  'colsample_bytree': 0.9})

In [41]:
# xg_study.best_trial.params =

In [42]:
lgb_study = optuna.create_study(
    direction='maximize',
    sampler= sampler
)

lgb_objective_func = LGB_Objective(train_ft, target, SEED)
lgb_study.optimize(lgb_objective_func, n_trials=100)

[I 2024-11-14 13:44:15,108] A new study created in memory with name: no-name-86ff4d26-faaa-4e8a-aab8-c3b8549546c4
[I 2024-11-14 13:44:40,307] Trial 0 finished with value: 0.6968916020805914 and parameters: {'n_estimators': 83, 'learning_rate': 0.31, 'max_depth': 13, 'num_leaves': 36, 'min_child_samples': 5}. Best is trial 0 with value: 0.6968916020805914.
[I 2024-11-14 13:47:31,723] Trial 1 finished with value: 0.709519424276098 and parameters: {'n_estimators': 418, 'learning_rate': 0.22, 'max_depth': 13, 'num_leaves': 83, 'min_child_samples': 6}. Best is trial 1 with value: 0.709519424276098.
[I 2024-11-14 13:48:58,671] Trial 2 finished with value: 0.714251420218891 and parameters: {'n_estimators': 212, 'learning_rate': 0.04, 'max_depth': 15, 'num_leaves': 71, 'min_child_samples': 13}. Best is trial 2 with value: 0.714251420218891.
[I 2024-11-14 13:49:21,844] Trial 3 finished with value: 0.7105720797570861 and parameters: {'n_estimators': 78, 'learning_rate': 0.09999999999999999, 'max

In [43]:
lgb_study.best_value, lgb_study.best_trial.params

(0.7183323395908452,
 {'n_estimators': 364,
  'learning_rate': 0.09999999999999999,
  'max_depth': 4,
  'num_leaves': 57,
  'min_child_samples': 28})

In [44]:
# (0.8977595290050033,
#  {'n_estimators': 422,
#   'learning_rate': 0.03,
#   'max_depth': 12,
#   'num_leaves': 50,
#   'min_child_samples': 30,
#   'subsample': 0.8,
#   'colsample_bytree': 0.5,
#   'reg_alpha': 0.8,
#   'reg_lambda': 1.0})

# cv 점수 확인하기

In [45]:
cv = KFold(5, random_state=SEED, shuffle=True)

In [46]:
models = [
    #('lr', LogisticRegression(random_state=SEED, **lr_study.best_trial.params)),
    #('rf', RandomForestClassifier(random_state=SEED, **rf_study.best_trial.params)),
    ('xg', XGBClassifier(random_state=SEED, **xg_study.best_trial.params)),
    ('lgb', LGBMClassifier(random_state=SEED, **lgb_study.best_trial.params))
    ]

In [47]:
model_score = []

for model_name, model in models:
    scores = cross_val_score(model, train_ft, target, cv=cv, n_jobs=-1, scoring="f1_macro")
    model_score.append((model_name, scores.mean()))

In [48]:
model_score

[('xg', 0.7207173157280883), ('lgb', 0.7183323395908452)]

In [ ]:
model_score

# 모델 학습및 테스트 데이터 예측

In [ ]:
# from xgboost import XGBClassifier

# model = XGBClassifier(
#     random_state=SEED,
#     n_estimators=193,
#     learning_rate=0.15,
#     max_depth=3,
#     gamma=2.6,
#     subsample=0.7,
#     colsample_bytree=0.6
# )


- xg

In [49]:
model_xg = XGBClassifier(random_state=SEED, **xg_study.best_trial.params)

In [50]:
model_xg.fit(train_ft, target)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=1.8000000000000003, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.06999999999999999,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=180, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [54]:
pred_proba_xg = model_xg.predict_proba(test_ft)[:,1]
pred_proba_xg

array([0.12570485, 0.34353182, 0.28248897, ..., 0.06330268, 0.11712732,
       0.030559  ], dtype=float32)

In [52]:
pred_xg = model_xg.predict(test_ft)
pred_xg

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
test_ft.shape

(12225, 494)

# 저장

In [55]:
submit

,ID,target
0,test_0,0.5
1,test_1,0.5
2,test_2,0.5
3,test_3,0.5
4,test_4,0.5
...,...,...
12220,test_12220,0.5
12221,test_12221,0.5
12222,test_12222,0.5
12223,test_12223,0.5


In [56]:
submit["target"] = pred_proba_xg
submit

,ID,target
0,test_0,0.125705
1,test_1,0.343532
2,test_2,0.282489
3,test_3,0.831861
4,test_4,0.621984
...,...,...
12220,test_12220,0.693037
12221,test_12221,0.307432
12222,test_12222,0.063303
12223,test_12223,0.117127


In [58]:
submit.to_csv(f"{DATA_PATH}submit_optuna_proba_xg.csv",index=False)

In [59]:
submit["target"] = pred_xg
submit

,ID,target
0,test_0,0
1,test_1,0
2,test_2,0
3,test_3,1
4,test_4,1
...,...,...
12220,test_12220,1
12221,test_12221,0
12222,test_12222,0
12223,test_12223,0


In [60]:
submit.to_csv(f"{DATA_PATH}submit_optuna_xg.csv",index=False)

In [65]:
# pred_xg_train = model_xg.predict(train_ft)
# pred_xg_train

In [66]:
# from sklearn.metrics import f1_score
# f1_score(target, pred_xg_train, average='macro')

- lgb

In [61]:
model_lgb = LGBMClassifier(random_state=SEED, **lgb_study.best_trial.params)

In [62]:
model_lgb.fit(train_ft, target)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 5874, number of negative: 9066
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.061562 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 64538
[LightGBM] [Info] Number of data points in the train set: 14940, number of used features: 515
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.393173 -> initscore=-0.433995
[LightGBM] [Info] Start training from score -0.433995
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

LGBMClassifier(learning_rate=0.09999999999999999, max_depth=4,
               min_child_samples=28, n_estimators=364, num_leaves=57,
               random_state=42)

In [63]:
pred_proba_lgb = model_lgb.predict_proba(test_ft)[:,1]
pred_proba_lgb.shape

(12225,)

In [64]:
pred_lgb = model_lgb.predict(test_ft)
pred_lgb

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
test_ft.shape

(12225, 494)

# 저장

In [ ]:
submit

,ID,target
0,test_0,0.5
1,test_1,0.5
2,test_2,0.5
3,test_3,0.5
4,test_4,0.5
...,...,...
12220,test_12220,0.5
12221,test_12221,0.5
12222,test_12222,0.5
12223,test_12223,0.5


In [67]:
submit["target"] = pred_proba_lgb
submit

,ID,target
0,test_0,0.050070
1,test_1,0.310116
2,test_2,0.358061
3,test_3,0.798857
4,test_4,0.601251
...,...,...
12220,test_12220,0.727596
12221,test_12221,0.253114
12222,test_12222,0.042713
12223,test_12223,0.247080


In [68]:
submit.to_csv(f"{DATA_PATH}submit_optuna_proba_lgb.csv",index=False)

In [69]:
submit["target"] = pred_lgb
submit

,ID,target
0,test_0,0.0
1,test_1,0.0
2,test_2,0.0
3,test_3,1.0
4,test_4,1.0
...,...,...
12220,test_12220,1.0
12221,test_12221,0.0
12222,test_12222,0.0
12223,test_12223,0.0


In [70]:
submit.to_csv(f"{DATA_PATH}submit_optuna_lgb.csv",index=False)

In [71]:
# pred_lgb_train = model_lgb.predict(train_ft)
# pred_lgb_train

In [72]:
# from sklearn.metrics import f1_score
# f1_score(target, pred_lgb_train, average='macro')

In [73]:
#앙상블
pred = pred_proba_xg * 0.5 + pred_proba_lgb * 0.5
pred

array([0.08788731, 0.32682398, 0.32027522, ..., 0.05300783, 0.18210375,
       0.02290047])

In [76]:
pred_final = (pred > 0.44).astype(int)
pred_final.mean()

0.3983640081799591

# 예측 파일 저장후 컴피티션 페이지에 제출

In [ ]:
submit

,ID,target
0,test_0,0.5
1,test_1,0.5
2,test_2,0.5
3,test_3,0.5
4,test_4,0.5
...,...,...
12220,test_12220,0.5
12221,test_12221,0.5
12222,test_12222,0.5
12223,test_12223,0.5


In [ ]:
submit["target"] = pred
submit

,ID,target
0,test_0,0.102195
1,test_1,0.437204
2,test_2,0.326997
3,test_3,0.821938
4,test_4,0.655256
...,...,...
12220,test_12220,0.778495
12221,test_12221,0.298809
12222,test_12222,0.061625
12223,test_12223,0.193119


#업로드

In [77]:
submit.to_csv(f"{DATA_PATH}submit_새출발_튜닝_3_proba.csv",index=False)

In [78]:
submit['target'] = pred_final
submit

,ID,target
0,test_0,0
1,test_1,0
2,test_2,0
3,test_3,1
4,test_4,1
...,...,...
12220,test_12220,1
12221,test_12221,0
12222,test_12222,0
12223,test_12223,0


In [79]:
submit.to_csv(f"{DATA_PATH}submit_새출발_튜닝_3.csv",index=False)